## Todo

ArrayListe als Cython/Numpy erstellen:
mit 5 Listen

position, gesch, masse, pos_neu, geschw_neu



methoden als nogil funktionen umschreiben







queue implementieren

multiprocessor code schreiben

worker pool nutzen

In [ ]:
#Beispiel Listen zum Testen

python_position = [[149_597_890_000, 0,0], [0,0,0]]
python_geschwindigkeit = [[0, 29.786 * 10**3,0], [0,0,0]]
python_masse = [[5.97219 * 10 ** 24], [1.9889 * 10 ** 30]]
python_position_neu = []
python_geschwindigkeit_neu = []
python_position_list = []

In [ ]:
%load_ext Cython

In [ ]:
#Umwandlung aus Python Listen in Numpy Arrays

import numpy as np
position = np.array(python_position, dtype=np.float32)
geschwindigkeit = np.array(python_geschwindigkeit,dtype=np.float32)
masse = np.array(python_masse,dtype=np.float32)
position_neu = np.array(python_position_neu ,dtype=np.float32)
geschwindigkeit_neu = np.ndarray(python_geschwindigkeit_neu ,dtype=np.float32)

## Einzelne Cython Module

In [8]:
import os
os.mkdir('/tmp/cython_galaxy')

FileExistsError: [Errno 17] File exists: '/tmp/cython_galaxy'

In [9]:
%%file /tmp/cython_galaxy/abstand.pyx
cimport cython
#Abstands Funktion
#Abstand_Sonne = position[1] - position[0]

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Abstand (float [:, :] position_view , float[:] result ,int first, int second) nogil:   
    result[0] = position_view[first][0] - position_view[second][0]
    result[1] = position_view[first][1] - position_view[second][1]
    result[2] = position_view[first][2] - position_view[second][2]

Overwriting /tmp/cython_galaxy/abstand.pyx


In [28]:
%%file /tmp/cython_galaxy/betrag.pyx
cimport cython
from libc.math cimport sqrt


#Betrags Funktion
#Betrag_Erde  = sqrt( Abstand_Erde**2 + Abstand_Erde**2 + Abstand_Erde**2)
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Betrag (float [:] abstand_view , float[:] result) nogil:
    result[0] = sqrt(abstand_view[0]**2 + abstand_view[1]**2 + abstand_view[2]**2)

Overwriting /tmp/cython_galaxy/betrag.pyx


In [11]:
%%file /tmp/cython_galaxy/kraft.pyx
cimport cython

#Kraft Funktion
#Kraft_Erde = G * (masse_erde * masse_sonne / (Betrag_Erde  ** 3)) * Abstand_Erde
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Kraft (float [:] masse_view , float [:, :] abstand_view,  float Betrag\
                   ,int first_planet, int second_planet, float[:] result) nogil:
    cdef float G = 6.672 * 10 ** -11
    result[0] = G * (masse[first_planet] * masse[second_planet] / (Betrag  ** 3)) * abstand_view[0]
    result[1] = G * (masse[first_planet] * masse[second_planet] / (Betrag  ** 3)) * abstand_view[1]
    result[2] = G * (masse[first_planet] * masse[second_planet] / (Betrag  ** 3)) * abstand_view[2]

Writing /tmp/cython_galaxy/kraft.pyx


In [12]:
%%file /tmp/cython_galaxy/beschleunigung.pyx
cimport cython

#Beschleunigungs Funktion
#Beschleunigung_Erde  = Kraft_Erde  / masse
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Beschleunigung (float [:] kraft_view , float [:] masse_view, int massen_index, float[:] result) nogil:
    cdef float G = 6.672 * 10 ** -11
    result[0] = kraft_view[0]  / masse[massen_index]
    result[1] = kraft_view[1]  / masse[massen_index]
    result[2] = kraft_view[2]  / masse[massen_index]

Writing /tmp/cython_galaxy/beschleunigung.pyx


In [13]:
%%file /tmp/cython_galaxy/positions_aktualliersierung.pyx
cimport cython

#Positions Aktualliersierung
#position = position + dt * geschwindigkeit  + ((dt ** 2) / 2) * Beschleunigung_Erde
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Positions_aktualliersierung (float [:, :] position_view , float [:, :] geschwindigkeit_view, \
                   float [:] beschleunigung_view, float dt, int planet_index) nogil:
    
    position_view[planet_index][0] = position_view[planet_index][0]  + dt * geschwindigkeit_view[planet_index][0] \
                                                + ((dt ** 2) / 2) * beschleunigung_view[0]
        
    position_view[planet_index][1] = position_view[planet_index][1]  + dt * geschwindigkeit_view[planet_index][1] \
                                                + ((dt ** 2) / 2) * beschleunigung_view[0]
        
    position_view[planet_index][2] = position_view[planet_index][2]  + dt * geschwindigkeit_view[planet_index][2] \
                                                + ((dt ** 2) / 2) * beschleunigung_view[0]

Overwriting /tmp/cython_galaxy/positions_aktualliersierung.pyx


In [14]:
%%file /tmp/cython_galaxy/geschwindigkeit_aktualliersierung.pyx
cimport cython

#Geschwindigkeits Aktuallisierung
#geschwindigkeit[0][0]  += dt * Beschleunigung_Erde[0]
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Geschwindigkeit_aktualliersierung (float [:, :] geschwindigkeit_view , float [:] beschleunigung_view, \
                   float dt ,int planet_index) nogil:
    geschwindigkeit_view[planet_index][0] = geschwindigkeit_view[planet_index][0] + dt * beschleunigung_view[0]
    geschwindigkeit_view[planet_index][1] = geschwindigkeit_view[planet_index][1] + dt * beschleunigung_view[1]
    geschwindigkeit_view[planet_index][2] = geschwindigkeit_view[planet_index][2] + dt * beschleunigung_view[2]

Overwriting /tmp/cython_galaxy/geschwindigkeit_aktualliersierung.pyx


In [17]:
%%file /tmp/cython_galaxy/schleife.pyx
from cython.parallel import prange
from abstand import Abstand
from betrag import Betrag
from beschleunigung import Beschleunigung
from positions_aktualliersierung import Positions_aktualliersierung
from geschwindigkeit_aktualliersierung import Geschwindigkeit_aktualliersierung
import math
cimport cython

def Schleife (pos_list)
    dt = 60
    Jahr = 360 * 24 * 60

    for _ in prange (Jahr, nogil=True): 


        ## Ausrechnen

        cdef float [:] Abstand_Erde = np.array([0,0,0],  dtype=np.float32)
        cdef float [:] Abstand_Sonne = np.array([0,0,0],  dtype=np.float32)
        # Erde = 0, Sonne = 1
        Abstand(position, Abstand_Erde ,0, 1)
        Abstand(position, Abstand_Sonne ,1, 0)

        cdef float [:] Betrag_Erde = np.array([0],  dtype=np.float32)
        cdef float [:] Betrag_Sonne = np.array([0],  dtype=np.float32)   
        Betrag(Abstand_Erde , Betrag_Erde)
        Betrag(Abstand_Sonne , Betrag_Sonne)

        cdef float [:] Kraft_Erde = np.array([0,0,0],  dtype=np.float32)
        cdef float [:] Kraft_Sonne = np.array([0,0,0],  dtype=np.float32)  
        Kraft(masse ,Abstand_Erde, Betrag_Erde ,0, 1, Kraft_Erde)
        Kraft(masse ,Abstand_Sonne, Betrag_Sonne ,0, 1, Kraft_Sonne)

        cdef float [:] Beschleunigung_Erde = np.array([0,0,0],  dtype=np.float32)
        cdef float [:] Beschleunigung_Sonne = np.array([0,0,0],  dtype=np.float32)
        Beschleunigung(Kraft_Erde , masse, 0, Beschleunigung_Erde)
        Beschleunigung(Kraft_Sonne , masse, 1, Beschleunigung_Sonne)


        ## Aktualisieren

        cdef int Erde_index = 0
        Positions_aktualliersierung (position, geschwindigkeit, Beschleunigung_Erde,\
                                     dt, Erde_index)
        Geschwindigkeit_aktualliersierung (geschwindigkeit ,Beschleunigung_Erde, \
                       dt ,Erde_index)

        cdef int Sonne_index = 0
        Positions_aktualliersierung (position, geschwindigkeit, Beschleunigung_Sonne,\
                                     dt, Sonne_index)
        Geschwindigkeit_aktualliersierung (geschwindigkeit ,Beschleunigung_Erde, \
                       dt ,Sonne_index)

        pos_list.append(position[Erde_index])

Writing /tmp/cython_galaxy/schleife.pyx


In [16]:
%%file /tmp/cython_galaxy/start.py

from schleifen import Schleifen
Schleifen(python_position_list)

Overwriting /tmp/cython_galaxy/start.py


In [22]:
%%file /tmp/cython_galaxy/setup.py


# Aufruf: python3 setup.py build_ext --inplace
# Windows: zusaetzliche Option --compiler=mingw32
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
import numpy

ext_modules=[ Extension("abstand", ["abstand.pyx"],
        extra_compile_args=['-O3'], libraries=['m']),
              Extension("betrag", ["betrag.pyx"],
        #extra_compile_args=['-O3'], libraries=['m'],
        extra_compile_args=['-O3'], libraries=['m'])
            #include_dirs=[numpy.get_include()])
                        
            ]
             
setup( name = 'cython demo',
  cmdclass = {'build_ext': build_ext},
  ext_modules = ext_modules)



from abstand import Abstand
from betrag import Betrag
from beschleunigung import Beschleunigung
from positions_aktualliersierung import Positions_aktualliersierung
from geschwindigkeit_aktualliersierung import Geschwindigkeit_aktualliersierung


Overwriting /tmp/cython_galaxy/setup.py


In [29]:
%%bash
cd /tmp/cython_galaxy
python3 setup.py build_ext --inplace

running build_ext
skipping 'abstand.c' Cython extension (up-to-date)
cythoning betrag.pyx to betrag.c
building 'betrag' extension
gcc -pthread -B /opt/anaconda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/anaconda/include/python3.6m -c betrag.c -o build/temp.linux-x86_64-3.6/betrag.o -O3
gcc -pthread -shared -B /opt/anaconda/compiler_compat -L/opt/anaconda/lib -Wl,-rpath=/opt/anaconda/lib -Wl,--no-as-needed -Wl,--sysroot=/ build/temp.linux-x86_64-3.6/betrag.o -L/opt/anaconda/lib -lm -lpython3.6m -o /tmp/cython_galaxy/betrag.cpython-36m-x86_64-linux-gnu.so


Traceback (most recent call last):
  File "setup.py", line 27, in <module>
    from beschleunigung import Beschleunigung
ModuleNotFoundError: No module named 'beschleunigung'


In [ ]:
%%bash
cd /tmp/cython_galaxy_einzel_build
python3 start.py

In [ ]:
%timeit Kraft_Berechnung()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

plt.plot([x[0] for x in pos_list], [x[1] for x in pos_list])
plt.show()

## Cython Test

In [ ]:
%timeit Kraft_Berechnung()

## Versuch vom 30.11.2017



python listen von zb geschwindigkeit...

umwandeln in numpy.array oder numpy.ndarray ?

übergeben an die cython funktion (nogil)

zugriff innerhalb der cython funktion (memoryview?)



## Erster Numpy Test mit Listen als numpy Array

In [ ]:
import numpy as np
planet_position = [[149_597_890_000, 0,0], [0,0,0], [1,1,1]]
position = np.array(planet_position,  dtype=np.float32)

print (position)
print()
print(position[0][0])


## Memory View Beispiel

In [ ]:
%load_ext Cython

In [ ]:
%%cython -f -c-fopenmp --link-args=-fopenmp

import numpy as np

narr = np.arange(9, dtype=np.dtype("d")).reshape((3, 3))
cdef double [:, :] narr_view = narr

print (narr)
print (narr_view)
print (narr_view[2][2])

## Memory View Beispiel Test Umbau

In [ ]:
%%cython -f -c-fopenmp --link-args=-fopenmp

import numpy as np

planet_position = [[149_597_890_000, 0,0], [0,0,0], [1,1,1]]
position = np.array(planet_position,  dtype=np.float32)
cdef float [:, :] position_view = position
print (position)
print (position_view)
print (position_view[0][0])
print (position_view[2][2])

position_view[2][2] = 5
print (position_view[2][2])

## Erfolgreicher NoGil Test mit Memory View

In [ ]:
%%cython -f -c-fopenmp --link-args=-fopenmp

import numpy as np
cimport numpy as cnp
cimport cython
ctypedef cnp.float32_t dtype_t

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Abstand (float [:, :] position_view , float[:] result ,int first, int second) nogil:   
    result[0] = position_view[first][0] - position_view[second][0]
    result[1] = position_view[first][1] - position_view[second][1]
    result[2] = position_view[first][2] - position_view[second][2]

planet_position = [[149_597_890_000, 0,0], [0,0,0], [1,1,1]]
position = np.array(planet_position,  dtype=np.float32)
cdef float [:, :] position_view = position
cdef float [:] result1 = np.array([0,0,0],  dtype=np.float32)
cdef float [:] result2 = np.array([0,0,0],  dtype=np.float32)

Abstand(position_view, result1, 0, 1)
position_view_result1 = np.asarray(result1)

Abstand(position_view, result2, 1, 0)
position_view_result2 = np.asarray(result2)

print (position_view_result1)
print (position_view_result2)

## Build Test

In [1]:
import os
os.mkdir('/tmp/cython_galaxy_einzel_build')

FileExistsError: [Errno 17] File exists: '/tmp/cython_galaxy_einzel_build'

In [2]:
%%file /tmp/cython_galaxy_einzel_build/abstand.pyx
#Abstands Funktion
#Abstand_Sonne[0] = position[1][0] - position[0][0]
cimport cython

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Abstand (float [:, :] position_view , float[:] result ,int first, int second) nogil:   
    result[0] = position_view[first][0] - position_view[second][0]
    result[1] = position_view[first][1] - position_view[second][1]
    result[2] = position_view[first][2] - position_view[second][2]

Overwriting /tmp/cython_galaxy_einzel_build/abstand.pyx


In [3]:
%%file /tmp/cython_galaxy_einzel_build/abstand_test.pyx

import numpy as np
from abstand import Abstand 
cimport numpy as cnp
cimport cython
ctypedef cnp.float32_t dtype_t

def test():
    planet_position = [[149_597_890_000, 0,0], [0,0,0], [1,1,1]]
    position = np.array(planet_position,  dtype=np.float32)
    cdef float [:, :] position_view = position
    cdef float [:] result1 = np.array([0,0,0],  dtype=np.float32)
    cdef float [:] result2 = np.array([0,0,0],  dtype=np.float32)

    Abstand(position_view, result1, 0, 1)
    position_view_result1 = np.asarray(result1)

    Abstand(position_view, result2, 1, 0)
    position_view_result2 = np.asarray(result2)

    print (position_view_result1)
    print (position_view_result2)

Overwriting /tmp/cython_galaxy_einzel_build/abstand_test.pyx


In [4]:
%%file /tmp/cython_galaxy_einzel_build/start.py

from abstand_test import test
test()

Overwriting /tmp/cython_galaxy_einzel_build/start.py


In [5]:
%%file /tmp/cython_galaxy_einzel_build/setup.py


# Aufruf: python3 setup.py build_ext --inplace
# Windows: zusaetzliche Option --compiler=mingw32
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
import numpy

ext_modules=[ Extension("abstand", ["abstand.pyx"],
        extra_compile_args=['-O3'], libraries=['m']),
              Extension("abstand_test", ["abstand_test.pyx"],
        extra_compile_args=['-O3'], libraries=['m'],
            include_dirs=[numpy.get_include()])
            ]
             
setup( name = 'cython demo',
  cmdclass = {'build_ext': build_ext},
  ext_modules = ext_modules)


Overwriting /tmp/cython_galaxy_einzel_build/setup.py


## Build Script

In [6]:
%%bash
cd /tmp/cython_galaxy_einzel_build
python3 setup.py build_ext --inplace

running build_ext
cythoning abstand.pyx to abstand.c
building 'abstand' extension
gcc -pthread -B /opt/anaconda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/anaconda/include/python3.6m -c abstand.c -o build/temp.linux-x86_64-3.6/abstand.o -O3
gcc -pthread -shared -B /opt/anaconda/compiler_compat -L/opt/anaconda/lib -Wl,-rpath=/opt/anaconda/lib -Wl,--no-as-needed -Wl,--sysroot=/ build/temp.linux-x86_64-3.6/abstand.o -L/opt/anaconda/lib -lm -lpython3.6m -o /tmp/cython_galaxy_einzel_build/abstand.cpython-36m-x86_64-linux-gnu.so
cythoning abstand_test.pyx to abstand_test.c
building 'abstand_test' extension
gcc -pthread -B /opt/anaconda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/anaconda/lib/python3.6/site-packages/numpy/core/include -I/opt/anaconda/include/python3.6m -c abstand_test.c -o build/temp.linux-x86_64-3.6/abstand_test.o -O3
gcc -pthread -shared -B /o

In file included from /opt/anaconda/lib/python3.6/site-packages/numpy/core/include/numpy/ndarraytypes.h:1809:0,
                 from /opt/anaconda/lib/python3.6/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /opt/anaconda/lib/python3.6/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from abstand_test.c:485:
/opt/anaconda/lib/python3.6/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: Warnung: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it by " \
  ^~~~~~~


In [7]:
%%bash
cd /tmp/cython_galaxy_einzel_build
python3 start.py

[  1.49597897e+11   0.00000000e+00   0.00000000e+00]
[ -1.49597897e+11   0.00000000e+00   0.00000000e+00]
